# comparison with plume tracker outputs

because

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

from plot_utils import get_carree_axis, plot_particles

In [ ]:
TIJUANA_RIVER_DOMAIN = dict(
    S=32.528,
    N=32.71,
    W=-117.29,
    E=-117.11,
)
CLOSE_TIJUANA_DOMAIN = dict(
    S=32.53,
    N=32.5825,
    W=-117.162,
    E=-117.105,
)

In [ ]:
file_prefix = "pts_20200810_"
start = 30
incr = 100

In [ ]:
for i in range(start, 2400, incr):
    name = f"mat/{file_prefix}{str(i).zfill(4)}.mat"
    try:
        data = scipy.io.loadmat(name)
        plot_particles(data["yf"], data["xf"], None, CLOSE_TIJUANA_DOMAIN)
    except FileNotFoundError:
        print(f"file {name} not found")